In [1]:
from keras.models import *
from keras.layers import *
from keras.applications import *
from keras.preprocessing.image import *

Using TensorFlow backend.


In [2]:
import os

In [3]:
dataset_dir = '/dataset_dir'

In [4]:
train_dir = dataset_dir + '/train'
test_dir = dataset_dir + '/test'

In [5]:
def write_tfv(MODEL, image_size, lambda_func=None):
    width = image_size[0]
    height = image_size[1]
    input_tensor = Input((height, width, 3))
    x = input_tensor
    if lambda_func:
        x = Lambda(lambda_func)(x)
    
    base_model = MODEL(input_tensor=x, weights='imagenet', include_top=False)
    model = Model(base_model.input, GlobalAveragePooling2D()(base_model.output))
    
    print('\n')
    batch_size = 32
    gen = ImageDataGenerator()
    train_generator = gen.flow_from_directory(train_dir, image_size, shuffle=False, 
                                              batch_size=batch_size)
    test_generator = gen.flow_from_directory(test_dir, image_size, shuffle=False, 
                                             batch_size=batch_size, class_mode=None)
    
    print('generating training transfer-value')
    train = model.predict_generator(train_generator, train_generator.samples//batch_size+1)
    print('training set shape: {}'.format(train.shape))
    print('generating test transfer-value')
    test = model.predict_generator(test_generator, test_generator.samples//batch_size+1)
    print('test set shape: {}'.format(test.shape))
    
    filenames = train_generator.filenames
    for i, fname in enumerate(train_generator.filenames):
        filenames[i] = os.path.basename(fname).encode()
    print('start writing h5 file.')
    with h5py.File('tfv_{}.h5'.format(MODEL.__name__)) as h:
        h.create_dataset('train', data=train)
        h.create_dataset('test', data=test)
        h.create_dataset('label', data=train_generator.classes)
        h.create_dataset('train_filename', data=filenames)

In [40]:
write_tfv(ResNet50, (224, 224))



Found 22424 images belonging to 10 classes.
Found 79726 images belonging to 1 classes.
generating training transfer-value
training set shape: (22424, 2048)
generating test transfer-value
test set shape: (79726, 2048)
start writing h5 file.


In [41]:
write_tfv(Xception, (299, 299), xception.preprocess_input)



Found 22424 images belonging to 10 classes.
Found 79726 images belonging to 1 classes.
generating training transfer-value
training set shape: (22424, 2048)
generating test transfer-value
test set shape: (79726, 2048)
start writing h5 file.


In [6]:
write_tfv(InceptionV3, (299, 299), inception_v3.preprocess_input)

87908352/87910968 [============================>.] - ETA: 0s

Found 22424 images belonging to 10 classes.
Found 79726 images belonging to 1 classes.
generating training transfer-value
training set shape: (22424, 2048)
generating test transfer-value
test set shape: (79726, 2048)
start writing h5 file.


In [7]:
write_tfv(VGG16, (224, 224))

57548800/58889256 [============================>.] - ETA: 0s

Found 22424 images belonging to 10 classes.
Found 79726 images belonging to 1 classes.
generating training transfer-value
training set shape: (22424, 512)
generating test transfer-value
test set shape: (79726, 512)
start writing h5 file.


In [8]:
write_tfv(VGG19, (224, 224))

79265792/80134624 [============================>.] - ETA: 0s

Found 22424 images belonging to 10 classes.
Found 79726 images belonging to 1 classes.
generating training transfer-value
training set shape: (22424, 512)
generating test transfer-value
test set shape: (79726, 512)
start writing h5 file.
